# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
client_id = 'jAfnuXCOjnwcGn6qM_gtGQ'
api_key = 'F8zRphNUA3_KsClfINoGcYiMTsjEQ_BiscQNBmztrgRh3fEHf0X5HGmdptipmFZ95lmDoqWgUKWecQGaIsKQEjNdWR_M_Xn6lCh8MY0Ldj4v2M4M2mkNPd-NfDESXXYx'

import requests
term = 'Pizza'
location = 'Brooklyn NY'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "C8j0q4Ma_S5hBGuAI-aaww", "alias": "di-fara-pizza-brooklyn", "name": "Di Fara Pizza", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/Ga-HenQEAW_xd1gCX5IYfw/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/di-fara-pizza-brooklyn?adjust_creative=jAfnuXCOjnwcGn6qM_gtGQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jAfnuXCOjnwcGn6qM_gtGQ", "review_count": 3266, "categories": [{"alias": "pizza", "title": "Pizza"}], "rating": 4.0, "coordinates": {"latitude": 40.625093, "longitude": -73.961531}, "transactions": ["delivery", "pickup"], "price": "$$", "location": {"address1": "1424 Ave J", "address2": "", "address3": "", "city": "Brooklyn", "zip_code": "11230", "country": "US", "state": "NY", "display_address": ["1424 Ave J", "Brooklyn, NY 11230"]}, "phone": "+17182581367", "display_phone": "(718) 258-1367", "distance": 3559.746344303}, {"id": "ysqgdbSrezXgVwER2kQWKA", "alias": "julianas-pizza-brook

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [2]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [4]:
response.json()['total']

10400

In [6]:
import pandas as pd
df = pd.DataFrame(response.json()['businesses'])
df.head(10)

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,di-fara-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.625093, 'longitude': -73.961531}",(718) 258-1367,3559.746344,C8j0q4Ma_S5hBGuAI-aaww,https://s3-media4.fl.yelpcdn.com/bphoto/Ga-Hen...,False,"{'address1': '1424 Ave J', 'address2': '', 'ad...",Di Fara Pizza,+17182581367,$$,4.0,3266,"[delivery, pickup]",https://www.yelp.com/biz/di-fara-pizza-brookly...
1,julianas-pizza-brooklyn-5,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7026153030093, 'longitude': -7...",(718) 596-6700,7214.079409,ysqgdbSrezXgVwER2kQWKA,https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTx...,False,"{'address1': '19 Old Fulton St', 'address2': '...",Juliana's Pizza,+17185966700,$$,4.5,1988,[],https://www.yelp.com/biz/julianas-pizza-brookl...
2,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,6771.011722,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,434,"[delivery, pickup]",https://www.yelp.com/biz/l-industrie-pizzeria-...
3,barboncino-pizza-and-bar-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.67209, 'longitude': -73.95731}",(718) 483-8834,2664.990408,xRiLLXjeM2wOvVTYJH2__A,https://s3-media3.fl.yelpcdn.com/bphoto/48doX4...,False,"{'address1': '781 Franklin Ave', 'address2': '...",Barboncino Pizza & Bar,+17184838834,$$,4.0,899,[],https://www.yelp.com/biz/barboncino-pizza-and-...
4,joe-and-sals-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.6694466400113, 'longitude': -7...",(718) 484-8732,2555.936856,pQo-KWQxuH26y0IzZbQj0A,https://s3-media4.fl.yelpcdn.com/bphoto/V4YjC0...,False,"{'address1': '842 Franklin Ave', 'address2': '...",Joe & Sal's Pizza,+17184848732,$,4.5,99,"[delivery, pickup]",https://www.yelp.com/biz/joe-and-sals-pizza-br...
5,robertas-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.705043534338, 'longitude': -73...",(718) 417-1118,5881.757582,FhXjAc6nKLf414KxYujUHw,https://s3-media4.fl.yelpcdn.com/bphoto/xRNPKI...,False,"{'address1': '261 Moore St', 'address2': '', '...",Roberta's,+17184171118,$$,4.0,2676,"[delivery, pickup]",https://www.yelp.com/biz/robertas-brooklyn-2?a...
6,l-and-b-spumoni-gardens-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.594715, 'longitude': -73.981316}",(718) 449-1230,7319.894614,vyoA8dxwScuMV_AsTcjQcg,https://s3-media2.fl.yelpcdn.com/bphoto/BrTci9...,False,"{'address1': '2725 86th St', 'address2': '', '...",L & B Spumoni Gardens,+17184491230,$,4.0,3486,"[delivery, pickup]",https://www.yelp.com/biz/l-and-b-spumoni-garde...
7,yayas-pizzeria-new-york,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.64529, 'longitude': -73.94881}",(347) 627-8399,1111.963134,kymVU-N3-bTK5KA84ZvIxg,https://s3-media3.fl.yelpcdn.com/bphoto/G2JvCn...,False,"{'address1': '1675 Nostrand Ave', 'address2': ...",Yaya's Pizzeria,+13476278399,$,4.5,13,"[delivery, pickup]",https://www.yelp.com/biz/yayas-pizzeria-new-yo...
8,lucali-brooklyn-3,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.6818, 'longitude': -74.00024}",(718) 858-4086,6096.250479,Q9F2ocrmYuGt1yn3M7MOBw,https://s3-media1.fl.yelpcdn.com/bphoto/zpy57j...,False,"{'address1': '575 Henry St', 'address2': None,...",Lucali,+17188584086,$$,4.5,1339,[],https://www.yelp.com/biz/lucali-brooklyn-3?adj...
9,patsys-pizzeria-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.6818695, 'longitude': -73.976181}",(718) 622-2268,4520.468918,xrgzB0pvsxeGEIboE6I2gg,https://s3-media4.fl.yelpcdn.com/bphoto/1n5xL5...,False,"{'address1': '450 Dean St', 'address2': '', 'a...",Patsy's Pizzeria,+17186222268,$$,4.5,492,"[delivery, pickup]",https://www.yelp.

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
#Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!